In [1]:
import pandas as pd
import os

#prepare data and put into correct format for LostIslandBuildings
def prepareData():

    df = pd.read_csv('LostIslandBuildings.csv')

    #Rename Color / Area column to be Color
    df.rename(columns={'Color / Area': 'Color'}, inplace=True)

    #Separate Coordinates into X-Coord and Y-Coord
    df[['X-Coord', 'Y-Coord']] = df['Coordinates'].str.split(' ', expand=True)
    df = df.drop('Coordinates', axis=1)
    df['X-Coord'] = df['X-Coord'].str[2:]
    df['Y-Coord'] = df['Y-Coord'].str[2:]

    #Separate Buff Percent into categories
    buffCategories = ['HP Guardian', 'HP Shooter', 'HP Carrier', 'ATK Guardian', 'ATK Shooter', 'ATK Carrier', 'DEF Guardian', 'DEF Shooter', 'DEF Carrier']

    for i in range(len(buffCategories)):
        df[buffCategories[i]] = df['Buff Percent'].apply(lambda x: x.split(' ')[0] if buffCategories[i] in x else 0)

    #Separate Buff Percent into categories for Wonders
    wonderCondition = df['Alliance Influence'] == '25,000'
    wonderRowsOnly = df.loc[wonderCondition, :]
    wonderRowBuffs = ['3.6%', '3.6%', '3.6%', '9%', '9%', '9%', '9%', '9%', '9%']
    for i in range(len(buffCategories)):
        df.loc[wonderCondition, buffCategories[i]] = wonderRowBuffs[i]
    df.loc[wonderCondition, buffCategories] = wonderRowsOnly[buffCategories]

    df = df.drop('Buff Percent', axis=1)

    #Separate Soldier Production into categories
    productionCategories = ['Guardian', 'Shooter', 'Carrier']

    for i in range(len(productionCategories)):
        df[productionCategories[i] + ' Output'] = df['Soldier Production'].apply(lambda x: x.split(' ')[0] if productionCategories[i] in x else 0)

    #Separate Soldier Production for Wonders
    wonderRowProduction = ['44/h', '44/h', '44/h']
    productionCategoriesRenamed = ['Guardian Output', 'Shooter Output', 'Carrier Output']
    for i in range(len(productionCategoriesRenamed)):
        wonderRowsOnly[productionCategoriesRenamed[i]] = wonderRowProduction[i]
    df.loc[wonderCondition, productionCategoriesRenamed] = wonderRowsOnly[productionCategoriesRenamed]

    df = df.drop('Soldier Production', axis=1)

    #Rename Buildings to Building
    df.rename(columns={'Buildings': 'Building'}, inplace=True)

    #Formatting
    df = df.apply(lambda x: x.str.replace('%', '', regex=True))
    df = df.apply(lambda x: x.str.replace('/h', '', regex=True))
    df = df.apply(lambda x: x.str.replace(',', '', regex=True))
    df = df.fillna(0)

    df['Alliance Influence'] = df['Alliance Influence'].astype('int')
    for i in range(len(buffCategories)):
        df[buffCategories[i]] = df[buffCategories[i]].astype(float)
    for i in range(len(productionCategoriesRenamed)):
        df[productionCategoriesRenamed[i]] = df[productionCategoriesRenamed[i]].astype('int')

    df.to_csv('UpdatedLostIslandBuildings.csv', index=False)

In [1]:
import sqlite3 as sq3

#Convert csv file to SQLite database
def convertToSQLiteDB():
    updatedDf = pd.read_csv('UpdatedLostIslandBuildings.csv')
    conn = sq3.connect('LostIslandBuildings.db')
    updatedDf.to_sql('Buildings', conn, if_exists='replace', index=False)
    conn.close()

In [6]:
import datetime as date
import json
import os

#Create empty SQLite DB with only the headers initialized for a new Lost Island and setup textfile with alliance names
def initializeNewIsland(pinkName="", blueName="", brownName="", greenName="", purpleName="", yellowName="", tealName="", orangeName=""):
    current_date = date.datetime.now().strftime("%Y-%m-%d")
    save_path = "Events/" + current_date
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    #Connect to the sq3 db
    conn = sq3.connect(save_path + "/LostIslandAllianceBuildings.db")
    #Create cursor obj to exec SQL commands
    cursor = conn.cursor()

    #Table schema
    table_schema = """
    CREATE TABLE IF NOT EXISTS alliance_buildings (
        id INTEGER PRIMARY KEY,
        Alliance TEXT,
        Building TEXT,
        Color TEXT,
        "Alliance Influence" INTEGER,
        "X-Coord" INTEGER,
        "Y-Coord" INTEGER,
        "HP Guardian" FLOAT,
        "HP Shooter" FLOAT,
        "HP Carrier" FLOAT,
        "ATK Guardian" FLOAT,
        "ATK Shooter" FLOAT,
        "ATK Carrier" FLOAT,
        "DEF Guardian" FLOAT,
        "DEF Shooter" FLOAT,
        "DEF Carrier" FLOAT,
        "Guardian Output" INTEGER,
        "Shooter Output" INTEGER,
        "Carrier Output" INTEGER
    );"""
    cursor.execute(table_schema)
    conn.commit()
    conn.close()

    #Create config file
    islandConfig = save_path + "/config.json"

    islandConfigData = {
        "date": current_date,
        "alliances": {
            "pink": pinkName,
            "blue": blueName,
            "brown": brownName,
            "green": greenName,
            "purple": purpleName,
            "yellow": yellowName,
            "teal": tealName,
            "orange": orangeName
        }
    }

    with open(islandConfig, 'w') as configFile:
        json.dump(islandConfigData, configFile)
        
#testing
#initializeNewIsland("EOA", "", "LAO", "SEE", "SeN", "", "TAO")